In [24]:
import torch
import pickle
import json
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.nn as nn
from matplotlib import pyplot as plt
import spacy
import re
import nltk

nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to /Users/haileywu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
dirname = '../Dataset_v3/blog/blogtext.csv'
blog = pd.read_csv(dirname)
print("blog information: {0}".format(blog.shape))

blog information: (681284, 7)


In [4]:
blog = blog[['text']]

In [5]:
blog['wordlength'] = blog['text'].str.split().str.len() >= 50

In [6]:
blog_f = blog[blog['wordlength']]

In [7]:
blog_f = blog_f[['text']].reset_index(drop=True)

In [8]:
blog_f

text
0                  In het kader van kernfusie op aarde...
1                    Thanks to Yahoo!'s Toolbar I can ...
2                    I had an interesting conversation...
3                    Somehow Coca-Cola has a way of su...
4                    If anything, Korea is a country o...
...                                                   ...
474726         Dear Susan,  There once was a bird the ...
474727         Dear Susan, you keep asking me who your...
474728         Dear Susan,  'I have the second yeast i...
474729         Dear Susan:    Just to clarify, I am as...
474730         Hey everybody...and Susan,  You might a...

[474731 rows x 1 columns]

In [9]:
blog_f['Label'] = "0"
blog_f = blog_f[['text','Label']]

print(blog_f.shape[0])
blog_f.Label.describe()

474731


count     474731
unique         1
top            0
freq      474731
Name: Label, dtype: object

In [12]:
blog_f = blog_f[:2000]

In [15]:
# apply on the df['Text']
# retain all columns and add addtional columns spacy_words
def NER_replace(df):
    df['spacy_words'] = df['text'].apply(lambda x: list(nlp(x).ents))
    for index, row in df.iterrows():
        thistext = row.text
        for ent in row.spacy_words:
            if ent.label_ in ['PERSON','FAC','GPE','LOC','ORG']:
                thistext = thistext.replace(ent.text,ent.label_)
        df.at[index, 'text'] = thistext
    return df

In [16]:
blog_f1 = NER_replace(blog_f)

In [17]:
blog_f1.text[3]

"             PERSON has a way of summing up things so well.  In the early 1970s they had as their flagship jingle 'I'd Like to Buy the World a ORG' (to the tune of 'I'd Like to Teach the World to Sing') that pretty much summed up the post-Woodstock era so well.  It didn't add much to sales, but it was a catchy tune.  In GPE ORG's theme is  urlLink Stop Thinking. Feel it.  which pretty much sums up a lot about GPE and GPEns.  (Look at how relaxed that couple is, now that they stopped thinking and started feeling.) Of course they have a high regard for education and math and logic and such, but deep down I think many GPEns really like to work on emotion more than anything else.  Westerners seem to sublimate this moreso, or at least display it in a different way.  Maybe scratch all that...Westerners and GPEns are probably pretty similar, but the context in which we do it is different.  Anyways, if you think you're losing it in GPE just repeat to yourself 'Stop thinking, feel it. Stop thi

In [18]:
for ent in blog_f1.spacy_words[3]:
    if ent.label_ in ['PERSON','FAC','GPE','LOC','ORG']:
        print(ent.text, ent.start_char, ent.end_char,
          ent.label_)

Somehow Coca-Cola 13 30 PERSON
Coke 154 158 ORG
Coke 340 344 ORG
Korea 428 433 GPE
Korea 961 966 GPE


In [21]:
blog_f1['Text'] = blog_f1['text']
blog_f1 = blog_f1[['Text','Label']]

In [34]:
blog_sentences = []
indexes = []
for index, row in blog_f1.iterrows():
    sentences = tokenizer.tokenize(row.Text) 
    blog_sentences.extend(sentences)
    thisin = [index] * len(sentences)
    indexes.extend(thisin)
    
blog_sent_f = pd.DataFrame(
    {'Text': blog_sentences,
     'Index': indexes
    })

In [35]:
blog_sent_f['Label']=0

In [40]:
blog_sent_f.shape

(34033, 3)

In [41]:
blog_sent_f.to_csv("../Dataset_v3/blog/blog_2000_sent.csv")